# Text segmentation using Hidden Markov Models

### Theorical Questions

_1. Give the value of the $\pi$ vector of the initial probabilities_

Each mail actually contains a header, and the decoding necessarily begins in the state 1. Therefore, the initial probabilities are given by 

$$\pi = [1, 0]$$

2.  _What is the probability to move from state 1 to state 2 ? What is the probability to remain in state 2 ? What is the lower/higher probability ? Try to explain why_

The probability to move from state 1 to state 2 is given by the transition matrix $A$ as $A_{12} = 0.000781921964187974$. 

The probability to remain in state 2 is given by $A_{22} = 0.999218078035812$. 

The higher probability is the probability to remain in state 2. This is because the state 2 is the state that represents the body, and the state 1 represents the header. And it's impossible to have a mail with a header after the body.

3. What is the size of B ?

The size of B is $2 \times N$.

### Importing Libraries

In [30]:
import os
import glob
import numpy as np

In [31]:
ROOT = os.path.abspath('.')

PERL_DIR = os.path.join(ROOT,'PerlScriptAndModel')
RES_DIR = os.path.join(ROOT,'res')

### Coding/Decoding Mails

In [32]:
DATA_DIR = os.path.join(ROOT,'dat')

# Iterate through files and load the text 
def files_iter(data_dir, with_name=False):
    files = glob.glob('{}/*.dat'.format((data_dir)))
    if with_name:
        for f in files:
            # Get the filename 
            name = f.split('\\')[-1].split('.')[0]
            # Return filename and associated text
            yield name, open(f).read()
    else:
        for f in files :
            yield open(f).read()

In [33]:
# And we get a generator that will allow us to iterate through the mails
mail_iter = files_iter(DATA_DIR, with_name=True)

In [34]:
for name, mail in mail_iter:
    print(name, mail)
    break

mail1 70
114
111
109
32
101
120
109
104
45
119
111
114
107
101
114
115
45
97
100
109
105
110
64
114
101
100
104
97
116
46
99
111
109
32
32
84
104
117
32
65
117
103
32
50
50
32
49
50
58
51
54
58
50
51
32
50
48
48
50
10
82
101
116
117
114
110
45
80
97
116
104
58
32
60
101
120
109
104
45
119
111
114
107
101
114
115
45
97
100
109
105
110
64
115
112
97
109
97
115
115
97
115
115
105
110
46
116
97
105
110
116
46
111
114
103
62
10
68
101
108
105
118
101
114
101
100
45
84
111
58
32
122
122
122
122
64
108
111
99
97
108
104
111
115
116
46
110
101
116
110
111
116
101
105
110
99
46
99
111
109
10
82
101
99
101
105
118
101
100
58
32
102
114
111
109
32
108
111
99
97
108
104
111
115
116
32
40
108
111
99
97
108
104
111
115
116
32
91
49
50
55
46
48
46
48
46
49
93
41
10
9
98
121
32
112
104
111
98
111
115
46
108
97
98
115
46
110
101
116
110
111
116
101
105
110
99
46
99
111
109
32
40
80
111
115
116
102
105
120
41
32
119
105
116
104
32
69
83
77
84
80
32
105
100
32
68
48
51
69
53
52
51
67
51
54
10
9
102
111
1

### Distribution files

In [35]:
PERL_DIR = os.path.join(ROOT,'PerlScriptAndModel')

# Writing a function to get the probability data
def get_emission_prob(perl_dir):
    return np.loadtxt(os.path.join(perl_dir, 'P.text')).T

In [36]:
# Inputs to the Viterbi function
trans = np.array([[0.999218078035812, 0.000781921964187974],[0, 1]])
emission_prob = get_emission_prob(PERL_DIR)
states = ['H', 'B']
start_prob = np.array([0, 1])

### To implement:

In [37]:
# Viterbi function
def viterbi(obs, states, start_prob, trans, emission_prob):
    """
        Viterbi Algorithm Implementation

        Keyword arguments:
            - obs: sequence of observation
            - states:list of states
            - start_prob:vector of the initial probabilities
            - trans: transition matrix
            - emission_prob: emission probability matrix
        Returns:
            - seq: sequence of state
    """

    # Avoid underflow: use the logarithm !
    # Avoid 0 in logarithm: use a small constant !
    small = 1e-100

    log_start_prob = np.log(start_prob + small)
    log_trans = np.log(trans + small).T
    log_emission_prob = np.log(emission_prob + small)

    T = len(obs)  # Number of observations
    N = len(states)  # Number of model states

    # Initialisation
    log_P = np.zeros((T, N))
    Q = np.zeros((T, N))

    # Viterbi
    # Forward loop:
    for state in range(N):
        log_P[0, state] = log_start_prob[state] + log_emission_prob[state, obs[0]]
    
    for t in range(1, T):
        for j in range(N):
            Q[t, j] = np.argmax(log_P[t - 1, :] + log_trans[:, j])
            log_P[t, j] = np.max(log_P[t - 1, :] + log_trans[:, j] + log_emission_prob[j, obs[t]])

    # Backward loop
    path = np.zeros(T, dtype=int)
    path[-1] = np.argmax(log_P[-1, :])
    for i in range(T - 2, -1, -1):
        path[i] = Q[i + 1, path[i + 1]]
    return path

In [38]:
RES_DIR = os.path.join(ROOT,'res')

# Creating a directory to put the result of the viterbi function
if not os.path.exists(RES_DIR):
    os.mkdir(RES_DIR)
    
# Function that will write a viterbi path for a mail in a dedicated result file
def create_viterbi_path_file(mail_name, viterbi_path):
    with open('{}/{}_path.txt'.format(RES_DIR, mail_name), 'w') as f: 
        f.write(''.join([str(c) for c in viterbi_path]))

In [39]:
# Using our generator, we get the mail names and data
for name_file, data in mail_iter:
    # Find out the viterbi path using viterbi
    data = [int(c) for c in data.split()]
    viterbi_path = viterbi(data, states, start_prob, trans, emission_prob)
    # Put it in the result file
    create_viterbi_path_file(name_file, viterbi_path)

### Visualizing segmentation

In [10]:
# Writing a function to go into the directory and execute the perl script "segment.pl" on the mail in the given path
def exec_perl_script(mail, path):
    res = !cd {PERL_DIR}; perl segment.pl {mail} {path}
    return res

# Writing a function getting the original mail, the result of viterbi, and applying the segmentation script
# Then putting the result
def segment_mail(mail_name, data_dir, output_dir):
    # Get the full path of the mail
    mail = os.path.join(data_dir, mail_name + '.dat')
    # Get the full path of the result
    path = os.path.join(output_dir, mail_name + '_path.txt')
    # Execute the visualization script
    formatted_mail = exec_perl_script(mail, path)
    # Get the results
    formatted_mail_text = formatted_mail[-1]
    # Go through the resulting text until the cutting line
    ...
    # If this was not the last line, return the text cut in to parts: header and body
    ...
    # If not, it's just a header
    ...

In [11]:
# Getting mails names
...
# Call the function and look at the result of segmentation
...